In [24]:
# read vcf data
import sys
import os
import re
import argparse
import pandas as pd
import numpy as np  
import gzip
import pysam
import io
from struct_lmm import StructLMM

In [25]:
HAP_P  = "dataPrepScripts/outputFromPlink_local_ancestry_pcs.csv"
PHE_P = "testPlink/synthetic_small_v1.pheno1"
SUBSET = "Phenotype(liability)"
SNV_P = "dataPrepScripts/SNVoutputFromPlink.txt"

In [26]:
HAPS = pd.read_csv(HAP_P, header=1, index_col=0)
PHE = pd.read_csv(PHE_P, sep="\t")
SNV = pd.read_csv(SNV_P, sep="\t")

In [27]:
y = PHE[SUBSET].to_numpy()

In [28]:
E = HAPS.iloc[:, 1:].to_numpy()

In [29]:
M = np.concatenate([np.ones((E.shape[0], 1)), E], axis=1)

In [30]:
g = SNV.iloc[:, 1].to_numpy()

In [31]:
y, M, E, g, y.shape, E.shape, M.shape, g.shape

(array([ 1.139105e+00,  1.939241e+00, -1.037053e+00,  1.150725e+00,
        -1.050160e-01,  3.408850e-01, -9.016580e-01,  7.595100e-02,
        -1.889157e+00, -4.663790e-01, -8.754610e-01,  4.759140e-01,
        -4.548790e-01, -1.099440e-01,  8.072180e-01,  1.308200e-01,
        -4.245300e-02,  2.244840e-01, -1.846019e+00,  1.131163e+00,
         3.207100e-01, -3.698770e-01,  6.610200e-01, -5.059360e-01,
        -1.672400e-02, -1.148400e+00,  1.338648e+00, -1.492279e+00,
        -2.123174e+00, -1.146319e+00,  8.552560e-01, -3.147860e-01,
         1.415500e-02, -1.800993e+00,  2.335370e-01, -1.226311e+00,
        -8.270370e-01,  3.683010e-01, -5.135070e-01,  1.276512e+00,
        -3.631930e-01, -6.862010e-01,  7.145840e-01,  5.497190e-01,
         1.083874e+00,  2.928810e-01,  1.921100e-01,  1.067058e+00,
        -2.395190e-01, -7.677600e-01,  1.913819e+00, -2.540447e+00,
         1.602938e+00, -5.130950e-01, -1.822552e+00,  2.668810e-01,
         7.798000e-02,  1.031718e+00,  9.974800e

In [ ]:
# %%time
lmm = StructLMM(y, M, E)
lmm.fit()
lmm.score_2dof_inter(g)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.48707D+02    |proj g|=  1.10000D+01

At iterate    1    f=  8.48652D+02    |proj g|=  2.17437D+00

At iterate    2    f=  8.48650D+02    |proj g|=  2.11976D-02

At iterate    3    f=  8.48650D+02    |proj g|=  3.88921D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      3      5      4     0     0   3.889D-05   8.486D+02
  F =   848.64982076633351     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
CPU time

7.30867574205967e-10